# CS246 - Colab 4
## Collaborative Filtering

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 198 kB 42.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=846c28d9891d9c0c4eb60629123b2ae376a1a83c8333f7b662a4bd6a1486d8ac
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-09-27 12:56:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  5.97MB/s    in 2.2s    

2021-09-27 12:56:58 (5.97 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [8]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [9]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [10]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [18]:
# YOUR CODE HERE
print("Number of ratings in the training dataset is {}".format(training.count()))
print("Number of movies in the training dataset is {}".format(items.count()))

Number of ratings in the training dataset is 80000
Number of movies in the training dataset is 1682


In [19]:
print("Number of ratings in the test dataset is {}".format(test.count()))

Number of ratings in the test dataset is 20000


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [17]:
# YOUR CODE HERE
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# Init Alternating Least Squares method 
als = ALS(maxIter=10, 
          regParam=0.01, 
          userCol="user_id", 
          itemCol="item_id", 
          ratingCol="rating",
          coldStartStrategy="drop")

# Fitting data
model = als.fit(training)

Now compute the RMSE on the test dataset.


In [21]:
# YOUR CODE HERE
# Predict
predictions = model.transform(test)

# Init evaluator
evaluator = RegressionEvaluator(metricName="rmse", 
                                labelCol="rating",
                                predictionCol="prediction")
# Calculate rmse
rmse = evaluator.evaluate(predictions)
print("Root Mean Square Error is {}".format(rmse))

Root Mean Square Error is 1.1397346047346344


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [22]:
# YOUR CODE HERE
# top-k recommendations
top_k = 5

# Generate top k movie recommendations for each user
userRecom = model.recommendForAllUsers(top_k)
# Generate top k user recommendations for each movie
movieRecom = model.recommendForAllItems(top_k)

In [23]:
userRecom.printSchema()

root
 |-- user_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- item_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [24]:
movieRecom.printSchema()

root
 |-- item_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- user_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [25]:
userRecom.show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|    471|[{1426, 13.288593...|
|    463|[{1311, 6.442218}...|
|    833|[{253, 6.427563},...|
|    496|[{1205, 7.4418244...|
|    148|[{916, 14.056533}...|
|    540|[{1368, 6.1685133...|
|    392|[{1194, 6.3285866...|
|    243|[{1643, 6.5646343...|
|    623|[{634, 8.03973}, ...|
|    737|[{902, 8.882012},...|
|    897|[{1615, 6.7017984...|
|    858|[{700, 10.4833145...|
|     31|[{1154, 9.590729}...|
|    516|[{958, 8.63222}, ...|
|    580|[{1286, 8.735178}...|
|    251|[{1643, 5.753018}...|
|    451|[{1159, 8.209106}...|
|     85|[{1643, 5.6440306...|
|    137|[{1643, 9.382638}...|
|    808|[{308, 8.384169},...|
+-------+--------------------+
only showing top 20 rows



In [26]:
movieRecom.show()

+-------+--------------------+
|item_id|     recommendations|
+-------+--------------------+
|   1580|[{914, 1.2961805}...|
|    471|[{89, 5.5014544},...|
|   1591|[{153, 10.453026}...|
|   1342|[{211, 4.124916},...|
|    463|[{614, 7.0540915}...|
|    833|[{93, 8.456303}, ...|
|   1645|[{811, 6.8285913}...|
|    496|[{820, 7.3802214}...|
|    148|[{364, 5.770133},...|
|   1088|[{726, 7.033022},...|
|   1238|[{202, 7.2633214}...|
|    540|[{39, 7.043648}, ...|
|   1460|[{762, 5.3364077}...|
|    392|[{340, 7.0916615}...|
|   1522|[{153, 5.7960777}...|
|    243|[{471, 5.413481},...|
|    623|[{39, 5.294245}, ...|
|   1483|[{39, 13.604742},...|
|   1084|[{475, 7.5870833}...|
|   1025|[{335, 7.8934298}...|
+-------+--------------------+
only showing top 20 rows



In [27]:
'''
FieldA    FieldB    ArrayField
1         A         {1,2,3}
2         B         {3,5}

Explode =>

FieldA    FieldB    ExplodedField
1         A         1
1         A         2
1         A         3
2         B         3
2         B         5
'''
explode_userRecom = userRecom.withColumn("recommendations", 
                                    explode(col("recommendations"))).select('user_id', 
                                                                            'recommendations.item_id',
                                                                            'recommendations.rating')
explode_userRecom.show()

+-------+-------+---------+
|user_id|item_id|   rating|
+-------+-------+---------+
|    471|   1426|13.288593|
|    471|    916|11.171058|
|    471|   1154|10.366758|
|    471|   1121|10.161558|
|    471|   1311| 9.955751|
|    463|   1311| 6.442218|
|    463|   1205|6.4086804|
|    463|    958|5.7549186|
|    463|   1479| 5.516664|
|    463|    962| 5.512805|
|    833|    253| 6.427563|
|    833|   1113|6.0544386|
|    833|    115| 5.972625|
|    833|   1128| 5.785963|
|    833|   1496| 5.532019|
|    496|   1205|7.4418244|
|    496|   1120|7.1017704|
|    496|   1005| 6.872428|
|    496|    583| 6.643388|
|    496|    115|6.5394607|
+-------+-------+---------+
only showing top 20 rows



In [28]:
joint_explode_userRecom = explode_userRecom.join(items, explode_userRecom.item_id == items.item_id)
joint_explode_userRecom.show()

+-------+-------+---------+-------+--------------------+
|user_id|item_id|   rating|item_id|               movie|
+-------+-------+---------+-------+--------------------+
|    471|   1426|13.288593|   1426|Grace of My Heart...|
|    471|    916|11.171058|    916|Lost in Space (1998)|
|    471|   1154|10.366758|   1154|   Alphaville (1965)|
|    471|   1121|10.161558|   1121|Umbrellas of Cher...|
|    471|   1311| 9.955751|   1311|Waiting to Exhale...|
|    463|   1311| 6.442218|   1311|Waiting to Exhale...|
|    463|   1205|6.4086804|   1205|Secret Agent, The...|
|    463|    958|5.7549186|    958|To Live (Huozhe) ...|
|    463|   1479| 5.516664|   1479|     Reckless (1995)|
|    463|    962| 5.512805|    962|Ruby in Paradise ...|
|    833|    253| 6.427563|    253|Pillow Book, The ...|
|    833|   1113|6.0544386|   1113|Mrs. Parker and t...|
|    833|    115| 5.972625|    115|Haunted World of ...|
|    833|   1128| 5.785963|   1128|Heidi Fleiss: Hol...|
|    833|   1496| 5.532019|   1

Once you have working code for each cell above, **head over to Gradescope, carefully read the questions, and submit your solution for this Colab**!